In [1]:
R.<y>=QQ[]

In [2]:
presnost=200

In [3]:
CC=ComplexField(presnost)

In [4]:
def skolske_nasobeni_pol(f,g):
    n = f.degree()
    m = g.degree()
    res = 0
    for i in range(n+m+1):
        coef_sum = 0
        for j in range(max(0,i-m),min(i,n)+1):
            coef_sum += f[j]*g[i-j]
        res +=(f.args()[0]^i)*coef_sum
    return res

""" Vydel polynomy f a g se zbytkem
Vrati dvojici q,r, kde q je podíl a r zbyte splnujici deg(r) < deg(g); r = f-qg
"""
def skolske_deleni_pol(f,g):
    if g == 0:
        raise ZeroDivisionError("division by zero polynomial")
    res = 0
    while f.degree() >= g.degree():
        lm = (f.lc() / g.lc())*f.args()[0]^(f.degree()-g.degree()) #f.args()[0] returns the polynomial variable
        res += lm
        f = f - skolske_nasobeni_pol(lm,g)
    return res,f

In [5]:
"""Spočítá FFT, viz skripta
-- n: počet koeficientů
-- omega: příslušná n-tá odmocnina z 1
-- coeffs: koeficienty 
"""

def FFT(n,omega,coeffs):
    if n == 1:
        return coeffs
    n2 = n//2
    b = FFT(n2, omega*omega,coeffs[::2]) #Alternativně coeffs[::2] [coeffs[2*i] for i in range(n2)]
    c = FFT(n2, omega*omega,coeffs[1::2]) #coeffs[1::2] [coeffs[2*i+1] for i in range(n2)]
    res = [0 for i in range(n)]
    om_pow = 1
    for i in range(n2):
        temp = om_pow*c[i]
        res[i] = b[i]+temp
        res[n2+i] = b[i]-temp
        om_pow *= omega
    return res
        
#Inverzní FFT
def IFFT(n,omega,coeffs):
    inv = omega^(-1)
    return [(omega.parent()(n))^(-1)*elem for elem in FFT(n,inv,coeffs)] 
#omega.parent() najde okruh ze kterého je omega, aby v něm mohla invertovat n

In [6]:
"""Funkce pro násobení dvou celočíselných polynomů za použítí komplexních odmocnin z 1
-- f,g: libovolné polynomy
-- prec: počet bitů s jejichž přesností se počítá v komplexních číslech
"""
def mul_complex(f,g, prec = 64):
    C = ComplexField(prec)
    n = f.degree()+g.degree()+1
    n = 1 << n.nbits() #dosadí do n nejbližší větší mocninu 2 než (stupeň f + stupeň g)
    
    f_coeffs = f.coefficients(sparse=False)+[0] * (n - f.degree()-1) #doplní nulami koeficienty f a g do počtu n 
    g_coeffs = g.coefficients(sparse=False)+[0] * (n - g.degree()-1) #k získání koef. lze použít i např. f.list()
    omega = C(e)**(C(2.)*C(pi)*C(I)/C(n)) #n-tá odmocnina z 1
    F = FFT(n,omega,f_coeffs)
    G = FFT(n,omega,g_coeffs)
    RES = [F[i]*G[i] for i in range(n)]
    res = IFFT(n,omega,RES)
    #print(res[::-1])
    res = [item.real().round() for item in res] #vezme z komplexního čísla reálnou část a zaokrouhlí ji na nejbližší celé číslo
    return f.parent(res) #převede koeficienty zpátky do polynomu z původního oboru (ten je = f.parent())

In [7]:
# Vynásobí dva racionální polynomy za pomocí komplexní odmocniny z 1 a FFT
def rft(f,g, prec=presnost):
    if R(f).is_constant() or R(g).is_constant():
        return f*g
    a = f.denominator()
    b = g.denominator()
    ff = (a*f).change_ring(ZZ)
    gg = (b*g).change_ring(ZZ)
    res = mul_complex(ff,gg,prec).change_ring(QQ)
    return res/(a*b)


In [27]:
# Vynásobí dva racionální polynomy za pomocí modularneho nasobenia
def fast_mul_rat(f,g, prec=64):
    try:
        fstupen=f.degree()
        gstupen=g.degree()
    except:
        return f*g
    a = f.denominator()
    b = g.denominator()
    ff = (a*f).change_ring(ZZ)
    gg = (b*g).change_ring(ZZ)
    res = modular_fast_mul(ff,gg).change_ring(QQ)
    return res/(a*b)



#Modulárně za pomocí FFT vynásobí dva celočíselné polynomy
def modular_fast_mul(f,g):
    if f == 0 or g == 0:
        return f.parent(0)
    try:
        fstupen=f.degree()
        gstupen=g.degree()
    except:
        return f*g
    k = (fstupen+gstupen+1).nbits() #aby 2^k>f.degree()+g.degree()
    r = max(abs(max(f)),abs(min(f)))
    s = max(abs(max(g)),abs(min(g)))
    n = 2*(max(fstupen,gstupen)+1)*r*s
    p = find_suitable_prime(k,n)
    
    pow2 = 2^k 
    F = Zmod(p)
    S = F['x']
    fp = S(f) #polynomials in Zmod(p)[]
    gp = S(g)
    omega = F.multiplicative_generator() #finds pow2-th root of unity in Zmod(p)
    omega = omega^(omega.multiplicative_order()>>k)
    
    f_coeffs = fp.list()+[F(0)] * (pow2 - fp.degree()-1) #doplní nulami koeficienty f a g do počtu k
    g_coeffs = gp.list()+[F(0)] * (pow2 - gp.degree()-1)
    F = FFT(pow2,omega,f_coeffs)
    G = FFT(pow2,omega,g_coeffs)
    RES = [F[i]*G[i] for i in range(pow2)]
    res = IFFT(pow2,omega,RES)
    for i in range(len(res)):
        res[i] = ZZ(res[i])
        if res[i]> p//2:
            res[i] -= p #preved cleny zpatky do intervalu [-p//2,p//2]
    return f.parent(res)
    

# Najde prvočíslo p>n, že 2^k | p-1.
def find_suitable_prime(k,n):
    p = 1 << (n.nbits() - k) #najde nějaké číslo dost velké, aby p*2^k > n jako začátek
    pow2 = 1 << k #t = 2^k
    p=p*pow2+1 #tvar a*2^k+1
    
    while True:
        p+=pow2
        if p.is_prime():
            break
    return p


In [9]:
def inverzo(p,n):
    #print(p)
    try:
        p.is_constant() #skusi ci p je polynom (.is constant nie je implementovana pre racionalne cisla)
    except:
        return 1/p
    y=p.parent().gens()[0]     
    q=p.constant_coefficient()
    if q==0:
        raise Exception("neinvertovatelna vecicka")
    q=1/q
    #print(f"opakujem to tu {ZZ(n).nbits()+1} krat!!!")
    for i in range(1,ZZ(n).nbits()+1):
        q=(rft(q,(2-rft(p,q)))).truncate(2^(i))
    return R(q).truncate(n)

In [23]:
def inverzo_modular(p,n):
    #print(p)
    try:
        p.is_constant() #skusi ci p je polynom (.is constant nie je implementovana pre racionalne cisla)
    except:
        return 1/p
    y=p.parent().gens()[0]     
    q=p.constant_coefficient()
    if q==0:
        raise Exception("neinvertovatelna vecicka")
    q=1/q
    #print(f"opakujem to tu {ZZ(n).nbits()+1} krat!!!")
    for i in range(1,ZZ(n).nbits()+1):
        q=(fast_mul_rat(q,(2-fast_mul_rat(p,q)))).truncate(2^(i))
    return R(q).truncate(n)

def pol_div_modular(f,g):
    n=f.degree()
    m=g.degree()
    #op=lambda x: x.coefficients(sparse=false)[::-1]
    op=lambda x: x.reverse()
    if n<m:
        return 0,f  #vrati rovno vysledok
    y=f.parent().gens()[0]
    c=inverzo_modular(op(g),n-m+1)
    qo=fast_mul_rat(op(f),c).truncate(n-m+1)
    q=op(qo)
    q=q.shift(n-m-q.degree())
    return (q,f-rft(g,q))

In [10]:
def pol_div(f,g):
    n=f.degree()
    m=g.degree()
    #op=lambda x: x.coefficients(sparse=false)[::-1]
    op=lambda x: x.reverse()
    if n<m:
        return 0,f  #vrati rovno vysledok
    y=f.parent().gens()[0]
    c=inverzo(op(g),n-m+1)
    qo=rft(op(f),c).truncate(n-m+1)
    q=op(qo)
    q=q.shift(n-m-q.degree())
    return (q,f-rft(g,q))

In [11]:
dvoj_pol=lambda x,y:(R.random_element(x),R.random_element(y))

In [42]:

#funkcia na porovanie roznych funkcii na delenie alebo nasobenie dvoch polynomov
def porovnanie(stupen=10,stupen2=(1,10),opakovani=25,vypis=False):
    assert 0<stupen2[0] <= stupen2[1] <= stupen, 'stupen2 mimo rozsah'
    vytlac=lambda x:print(x) if vypis==True else 0
    vytlac(f'rychlost pre polynomy stupna {stupen}')
    funkcie=[skolske_deleni_pol,pol_div,skolske_nasobeni_pol,fast_mul_rat,rft]
    #funkcie=[skolske_deleni_pol,pol_div,pol_div_modular,skolske_nasobeni_pol,fast_mul_rat,rft]
    funkcied={i.__name__:i for i in funkcie}
    spravnostd=lambda x,y,f: any([i[0]==i[1] for i in zip(f(x,y),x.quo_rem(y))])
    spravnostn=lambda x,y,f: any([i[0]==i[1] for i in zip(f(x,y),x*y)])
    #print(ahoj[0])
    
    1+10*0.5           
    vysledok={i.__name__:[] for i in funkcie}
    for i in range(opakovani):
        dvojica=(stupen,randint(*stupen2))
        entita=dvoj_pol(*dvojica)
        vytlac(f'\t stupne: {stupen} : {dvojica[1]} ')
        for f in vysledok.keys():
            a,b=spravnostd(*entita,funkcied[f]),spravnostn(*entita,funkcied[f])
            cas=timeit(f'{f}(*{entita})',seconds=True)
            vytlac(f'{i}-ty pokus s funkciou "{f}" trval {cas}')
            if a or b:
                vytlac('\n\t vysledok bol spravny\n')
            else:
                vytlac('\t vysledok bol nespravny')
            vysledok[f].append(cas)      
    vysledok=dict(zip(vysledok,map(mean,vysledok.values())))
    for i in vysledok:
        vytlac(f'{i},\t {vysledok[i]}')
    vytlac('\n\n')
    vysledok=sorted(vysledok.items(),key=lambda x:x[1])
    if vypis:
        for i in range(len(funkcie)-1):
            nasobne=round(vysledok[i+1][1]/vysledok[i][1],5)
            vytlac(f'{vysledok[i][0]} je {nasobne} rychlejsie nez {vysledok[i+1][0]}')
    return vysledok

Funkcia porovnanie deli polynom stupna "stupen" nahodnym polynomom stupna z rozahu "stupnen2"
toto robi "opakovani" krat a potom casy spriemeruje a vypise

In [32]:
porovnanie(stupen=100,stupen2=(50,100),opakovani=1,vypis=True)

rychlost pre polynomy stupna 100
	 stupne: 100 : 75 
0-ty pokus s funkciou "skolske_deleni_pol" trval 0.05205428839981323

	 vysledok bol spravny

0-ty pokus s funkciou "pol_div" trval 0.04517690320026304

	 vysledok bol spravny

0-ty pokus s funkciou "skolske_nasobeni_pol" trval 0.01416775059995416

	 vysledok bol spravny

0-ty pokus s funkciou "fast_mul_rat" trval 0.006563978983991547

	 vysledok bol spravny

0-ty pokus s funkciou "rft" trval 0.011098856440003146

	 vysledok bol spravny

skolske_deleni_pol 0.05205428839981323
pol_div 0.04517690320026304
skolske_nasobeni_pol 0.01416775059995416
fast_mul_rat 0.006563978983991547
rft 0.011098856440003146
fast_mul_rat je 1.69087 rychlejsie nez rft
rft je 1.27651 rychlejsie nez skolske_nasobeni_pol
skolske_nasobeni_pol je 3.18871 rychlejsie nez pol_div
pol_div je 1.15223 rychlejsie nez skolske_deleni_pol


[('fast_mul_rat', 0.006563978983991547),
 ('rft', 0.011098856440003146),
 ('skolske_nasobeni_pol', 0.01416775059995416),
 ('pol_div', 0.04517690320026304),
 ('skolske_deleni_pol', 0.05205428839981323)]

In [37]:
porovnanie(stupen=50,stupen2=(1,50),opakovani=10,vypis=False)

[('skolske_nasobeni_pol', 0.002203001468802104),
 ('fast_mul_rat', 0.0024399671168002534),
 ('rft', 0.004355170802402426),
 ('skolske_deleni_pol', 0.014471904297589211),
 ('pol_div', 0.02306270548799512)]

In [39]:
porovnanie(stupen=10,stupen2=(1,10),opakovani=25,vypis=False)

[('skolske_nasobeni_pol', 0.00017265876416023825),
 ('skolske_deleni_pol', 0.00031986204825574534),
 ('fast_mul_rat', 0.0004068826113279211),
 ('rft', 0.001003667491519824),
 ('pol_div', 0.004010316379135357)]

Tu vidime, ze pre polynomy malych stupnov si moc nepomozeme

In [33]:
porovnanie(stupen=200,stupen2=(100,200),opakovani=3,vypis=False)

rychlost pre polynomy stupna 200
skolske_deleni_pol 0.23280836513343575
pol_div 0.1252749493332279
skolske_nasobeni_pol 0.0660699384000812
fast_mul_rat 0.02773029477335513
rft 0.02470766125336619


[('rft', 0.02470766125336619),
 ('fast_mul_rat', 0.02773029477335513),
 ('skolske_nasobeni_pol', 0.0660699384000812),
 ('pol_div', 0.1252749493332279),
 ('skolske_deleni_pol', 0.23280836513343575)]

pre polynomy podobne velkeho stupna usetrime viac casu

In [43]:
porovnanie(stupen=100,stupen2=(1,50),opakovani=30,vypis=False)

[('skolske_nasobeni_pol', 0.005723965104108792),
 ('fast_mul_rat', 0.006404999116002486),
 ('rft', 0.010381167442935598),
 ('pol_div', 0.0675950868066381),
 ('skolske_deleni_pol', 0.12077874009464963)]

In [41]:
porovnanie(stupen=100,stupen2=(99,100),opakovani=30,vypis=False)

[('skolske_deleni_pol', 0.0012193932838939756),
 ('fast_mul_rat', 0.010799050607199507),
 ('pol_div', 0.012891064289545368),
 ('rft', 0.013158801068003712),
 ('skolske_nasobeni_pol', 0.019473691507994474)]

In [45]:
porovnanie(stupen=500,stupen2=(450,500),opakovani=1,vypis=True)

rychlost pre polynomy stupna 500
	 stupne: 500 : 489 
0-ty pokus s funkciou "skolske_deleni_pol" trval 0.22993143540006714

	 vysledok bol spravny

0-ty pokus s funkciou "pol_div" trval 0.28755402120004875

	 vysledok bol spravny

0-ty pokus s funkciou "skolske_nasobeni_pol" trval 0.6162642104001861

	 vysledok bol spravny

0-ty pokus s funkciou "fast_mul_rat" trval 0.12791859200006

	 vysledok bol spravny

0-ty pokus s funkciou "rft" trval 0.07742526080000971

	 vysledok bol spravny

skolske_deleni_pol,	 0.22993143540006714
pol_div,	 0.28755402120004875
skolske_nasobeni_pol,	 0.6162642104001861
fast_mul_rat,	 0.12791859200006
rft,	 0.07742526080000971



rft je 1.65216 rychlejsie nez fast_mul_rat
fast_mul_rat je 1.79748 rychlejsie nez skolske_deleni_pol
skolske_deleni_pol je 1.25061 rychlejsie nez pol_div
pol_div je 2.14312 rychlejsie nez skolske_nasobeni_pol


[('rft', 0.07742526080000971),
 ('fast_mul_rat', 0.12791859200006),
 ('skolske_deleni_pol', 0.22993143540006714),
 ('pol_div', 0.28755402120004875),
 ('skolske_nasobeni_pol', 0.6162642104001861)]

In [46]:
porovnanie(stupen=500,stupen2=(450,500),opakovani=10,vypis=True)

rychlost pre polynomy stupna 500
	 stupne: 500 : 462 
0-ty pokus s funkciou "skolske_deleni_pol" trval 1.282496268599789

	 vysledok bol spravny

0-ty pokus s funkciou "pol_div" trval 0.4453755197999271
	 vysledok bol nespravny
0-ty pokus s funkciou "skolske_nasobeni_pol" trval 0.5980018908001512

	 vysledok bol spravny

0-ty pokus s funkciou "fast_mul_rat" trval 0.8334430655999313

	 vysledok bol spravny

0-ty pokus s funkciou "rft" trval 0.07780714639993676

	 vysledok bol spravny

	 stupne: 500 : 481 
1-ty pokus s funkciou "skolske_deleni_pol" trval 0.35029830359999325

	 vysledok bol spravny

1-ty pokus s funkciou "pol_div" trval 0.29877511399972717

	 vysledok bol spravny

1-ty pokus s funkciou "skolske_nasobeni_pol" trval 0.5808480325998971

	 vysledok bol spravny

1-ty pokus s funkciou "fast_mul_rat" trval 0.12610222379989863

	 vysledok bol spravny

1-ty pokus s funkciou "rft" trval 0.06968139679993328

	 vysledok bol spravny

	 stupne: 500 : 489 
2-ty pokus s funkciou "skolske

[('rft', 0.06951309001997288),
 ('fast_mul_rat', 0.2010463411400633),
 ('pol_div', 0.329866841519979),
 ('skolske_nasobeni_pol', 0.5767184926400296),
 ('skolske_deleni_pol', 0.6269648046600195)]